<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/experiment/only_one_objfunction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [247]:
!pip install autorch > log.txt
!pip install optuna > log.txt

In [248]:
import joblib
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import autorch
from autorch.function import sp2wt
import optuna

# LOAD DATA

In [249]:
icg_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/icg_col_names.pkl')
c620_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl')
c660_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl')
t651_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/t651_col_names.pkl')
c670_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl')

In [250]:
icg_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/icg_train.csv',index_col=0).dropna(axis=0)
c620_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_train.csv',index_col=0).dropna(axis=0)
c660_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c660_train.csv',index_col=0).dropna(axis=0)
c670_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c670_train.csv',index_col=0).dropna(axis=0)
t651_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/t651_train.csv',index_col=0).dropna(axis=0)
allidx = list(set(icg_df.index)&
      set(c620_df.index)&
      set(c660_df.index)&
      set(c670_df.index)&
      set(t651_df.index))
len(allidx)

1295

# INPUT

In [251]:
# icg
icg_input = icg_df.loc[allidx,icg_c['x']]
icg_input = icg_input.join(c620_df.loc[allidx,'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'])
icg_input = icg_input.join(c660_df.loc[allidx,'Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'])
icg_input = icg_input.join(c620_df.loc[allidx].filter(regex='Receiver Temp'))

# c620
c620_feed = c620_df.loc[allidx,c620_c['x41']]

# t651
t651_feed = t651_df.loc[allidx,t651_c['x41']]

# OUTPUT

In [252]:
c620_op = c620_df.loc[allidx,c620_c['density']+c620_c['yRefluxRate']+c620_c['yHeatDuty']+c620_c['yControl']]
c620_wt = c620_df.loc[allidx,c620_c['vent_gas_x']+c620_c['distillate_x']+c620_c['sidedraw_x']+c620_c['bottoms_x']]
c660_op = c660_df.loc[allidx,c660_c['density']+c660_c['yRefluxRate']+c660_c['yHeatDuty']+c660_c['yControl']]
c660_wt = c660_df.loc[allidx,c660_c['vent_gas_x']+c660_c['distillate_x']+c660_c['sidedraw_x']+c660_c['bottoms_x']]
c670_op = c670_df.loc[allidx,c670_c['density']+c670_c['yRefluxRate']+c670_c['yHeatDuty']+c670_c['yControl']]
c670_wt = c670_df.loc[allidx,c670_c['distillate_x']+c670_c['bottoms_x']]

# CONFIG

In [253]:
config = {
      # c620
      'c620_G':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_G.pkl',
      'c620_F':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_F.pkl',
      'c620_op_min':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c620_op_min.pkl',
      'c620_op_max':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c620_op_max.pkl',

      # c660
      'c660_G':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c660_G.pkl',
      'c660_F':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c660_F.pkl',
      'c660_op_min':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c660_op_min.pkl',
      'c660_op_max':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c660_op_max.pkl',

      # c670
      'c670_M':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c670.pkl',
      
      # col_names
      'icg_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/icg_col_names.pkl',
      'c620_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl',
      'c660_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl',
      'c670_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl',
      
      # Special column (0.9999 & 0.0001)
      'index_9999_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_9999.pkl',
      'index_0001_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_0001.pkl',

      # sp
      'c620_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c620_wt_always_same_split_factor_dict.pkl',
      'c660_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c660_wt_always_same_split_factor_dict.pkl',
      'c670_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c670_wt_always_same_split_factor_dict.pkl',

      'op_fix_model':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/fix_model.pkl',
      
          }

# 輸入與輸出的對應關係

In [254]:
# 對應關係1
a = icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]
b = c620_df['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']
a.join(b)

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
012-023,70.0,70.000008
001-008,90.0,90.000008
052-017,80.0,79.999992
127-023,70.0,70.000008
029-008,90.0,90.000000
...,...,...
112-020,70.0,70.000015
096-026,70.0,70.000015
059-026,70.0,70.000000
121-008,90.0,90.000008


In [255]:
#對應關係2
a = icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]
na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
b = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1)*10000
b.name = 'c660_wt_NA_in_Benzene_ppmw'
a.join(b)

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,c660_wt_NA_in_Benzene_ppmw
012-023,860.0,859.993319
001-008,920.0,920.004159
052-017,920.0,920.010937
127-023,950.0,949.970144
029-008,920.0,920.006208
...,...,...
112-020,980.0,979.995729
096-026,800.0,800.000302
059-026,920.0,920.001909
121-008,920.0,919.996604


In [256]:
#對應關係3
a = icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]
b = c660_df['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%']*10000
a.join(b)

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%
012-023,10.000074,10.000074
001-008,2.500000,2.500000
052-017,5.000856,5.000856
127-023,5.000000,5.000000
029-008,10.000004,10.000004
...,...,...
112-020,10.000116,10.000116
096-026,5.000002,5.000002
059-026,9.999970,9.999970
121-008,2.500000,2.500000


In [257]:
class AllSystem(object):
  def __init__(self,config):
    
    # C620 部份
    self.c620_G = joblib.load(config['c620_G'])
    self.c620_F = joblib.load(config['c620_F'])
    self.c620_op_min = joblib.load(config['c620_op_min'])
    self.c620_op_max = joblib.load(config['c620_op_max'])
    
    # C660 部份
    self.c660_G = joblib.load(config['c660_G'])
    self.c660_F = joblib.load(config['c660_F'])
    self.c660_op_min = joblib.load(config['c660_op_min'])
    self.c660_op_max = joblib.load(config['c660_op_max'])

    # C670 部份
    self.c670_M = joblib.load(config['c670_M'])
    
    # 修正模組修正模擬模跟現場數據間的差異
    self.op_fix_model = joblib.load(config['op_fix_model'])

    # 欄位名稱(全名)
    self.icg_col = joblib.load(config['icg_col_path'])
    self.c620_col = joblib.load(config['c620_col_path'])
    self.c660_col = joblib.load(config['c660_col_path'])
    self.c670_col = joblib.load(config['c670_col_path'])
    
    # 其他資訊
    self.c620_wt_always_same_split_factor_dict = joblib.load(config['c620_wt_always_same_split_factor_dict'])
    self.c660_wt_always_same_split_factor_dict = joblib.load(config['c660_wt_always_same_split_factor_dict'])
    self.c670_wt_always_same_split_factor_dict = joblib.load(config['c670_wt_always_same_split_factor_dict'])
    self.index_9999 = joblib.load(config['index_9999_path'])
    self.index_0001 = joblib.load(config['index_0001_path'])
    self.e = 1e-8 #小值

    # 廠區提供密度
    self.V615_density = 0.8626
    self.C820_density = 0.8731
    self.T651_density = 0.8749

    # 廠區通常調整欄位
    self.c620_op_col_can_change = ['Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC','Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC']
    self.c660_op_col_can_change = ['Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC','Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC']
  
  def inference(self,icg_input,c620_feed,t651_feed,real_data_mode=False):
    idx = icg_input.index
    # 計算c620_case
    c620_case = pd.DataFrame(index=idx,columns=self.c620_col['case'])
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'] = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values
    # 預測操作條件和分離係數
    c620_op = self.c620_G.predict(c620_case.join(c620_feed))
    
    c620_sp = self.c620_F.predict(c620_case.join(c620_feed).join(c620_op))
    # 計算組成
    s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
    
    #c660
    #計算c660_feed
    V615_Btm_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr'].values.reshape(-1,1)
    C820_Dist_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr'].values.reshape(-1,1)
    V615_Btm_ton = V615_Btm_m3*self.V615_density
    C820_Dist_ton = C820_Dist_m3*self.C820_density
    c620_feed_rate_ton = V615_Btm_ton+C820_Dist_ton
    c620_mf_side = np.sum(c620_feed_rate_ton*c620_feed.values*s3*0.01,axis=1,keepdims=True)
    c620_mf_bot = np.sum(c620_feed_rate_ton*c620_feed.values*s4*0.01,axis=1,keepdims=True)
    t651_mf = (icg_input['Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr']*self.T651_density).values.reshape(-1,1)
    c660_mf = t651_mf + c620_mf_side
    t651_mf_p ,c620_mf_side_p = t651_mf/c660_mf ,c620_mf_side/c660_mf
    c660_feed = c620_wt[self.c620_col['sidedraw_x']].values*c620_mf_side_p + t651_feed.values*t651_mf_p
    c660_feed = pd.DataFrame(c660_feed,index=idx,columns=self.c660_col['x41'])
    # 計算c660_case
    c660_case = pd.DataFrame(index=idx,columns=self.c660_col['case'])
    c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'] = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values
    c660_case['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'].values
    # 預測操作條件和分離係數
    c660_op = self.c660_G.predict(c660_case.join(c660_feed))
    
    c660_sp = self.c660_F.predict(c660_case.join(c660_feed).join(c660_op))
    # 計算組成
    s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])
    
    #c670
    # 計算c670_feed
    c660_mf_bot = np.sum(c660_mf*c660_feed.values*s4*0.01,axis=1,keepdims=True)
    c670_mf = c620_mf_bot + c660_mf_bot
    c620_mf_bot_p,c660_mf_bot_p = c620_mf_bot/c670_mf , c660_mf_bot/c670_mf
    c670_feed = c620_wt[self.c620_col['bottoms_x']].values*c620_mf_bot_p + c660_wt[self.c660_col['bottoms_x']].values*c660_mf_bot_p
    c670_feed = pd.DataFrame(c670_feed,index=idx,columns=self.c670_col['combined'])
    # 計算c670_upper_bf
    c670_bf = pd.DataFrame(index=idx,columns=self.c670_col['upper_bf'])
    c620_bot_x = c620_wt[self.c620_col['bottoms_x']].values
    c660_bot_x = c660_wt[self.c660_col['bottoms_x']].values
    upper_bf = (c660_bot_x*c660_mf_bot)/(c620_bot_x*c620_mf_bot+c660_bot_x*c660_mf_bot)
    upper_bf = pd.DataFrame(upper_bf,index=idx,columns=self.c670_col['upper_bf'])
    upper_bf[list(set(self.index_9999)&set(upper_bf.columns))] = 0.9999
    upper_bf[list(set(self.index_0001)&set(upper_bf.columns))] = 0.0001
    # 直接預測分離係數和操作條件
    c670_output = self.c670_M.predict(c670_feed.join(upper_bf))
    c670_sp,c670_op = c670_output.iloc[:,:41*2],c670_output.iloc[:,41*2:]    
    # 計算輸出組成
    s1 = c670_sp[self.c670_col['distillate_sf']].values
    s2 = c670_sp[self.c670_col['bottoms_sf']].values
    w1 = sp2wt(c670_feed,s1)
    w2 = sp2wt(c670_feed,s2)
    c670_wt = np.hstack((w1,w2))
    c670_wt = pd.DataFrame(c670_wt,index = idx,columns=self.c670_col['distillate_x']+self.c670_col['bottoms_x'])
    
    # 是否修正操作條件 for 現場數據
    if real_data_mode == False:
      return c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op
    
    if real_data_mode == True:
      # 有些欄位現場數據沒有
      c620_op_col = c620_op.drop(['Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      c660_op_col = c660_op.drop(['Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      c670_op_col = c670_op.drop(['Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      
      # 經過修正模組修正op
      op_pred = self.op_fix_model(torch.cat((
        torch.FloatTensor(c620_op[c620_op_col].values),
        torch.FloatTensor(c660_op[c660_op_col].values),
        torch.FloatTensor(c670_op[c670_op_col].values)),dim=1))
      op_pred = pd.DataFrame(op_pred.detach().numpy(),index=idx)
      
      # 新的op
      new_c620_op = op_pred.iloc[:,:8]
      new_c660_op = op_pred.iloc[:,8:16]
      new_c670_op = op_pred.iloc[:,-5:]
      new_c620_op.columns = c620_op_col
      new_c660_op.columns = c660_op_col
      new_c670_op.columns = c670_op_col
      
      # 更新op
      c620_op.update(new_c620_op)
      c660_op.update(new_c660_op)
      c670_op.update(new_c670_op)
      
      return c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op
  
  def recommend(self,icg_input,c620_feed,t651_feed,search_iteration=300,real_data_mode=False,auto_set_x0=False):
    
    # 先試算一遍
    idx = icg_input.index
    c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = self.inference(icg_input,c620_feed,t651_feed)
    
    # 是否要自動設置 70,980,10 , 否則的話使用者可以自行設定 例如,80,990,15 都可以
    if auto_set_x0 == True:
      icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'] = 70
      icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] = 980
      icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = 10
      print('===================auto_set_x0=====================================')
      print('icg_input[Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%] = 70')
      print('icg_input[Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw] = 980')
      print('icg_input[Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw] = 10')
      
    # c620 case初始設置
    c620_case = pd.DataFrame(index=idx,columns=self.c620_col['case'])
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'] = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values
    
    # c660 case初始設置
    c660_case = pd.DataFrame(index=idx,columns=self.c660_col['case'])
    c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'] = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values
    c660_case['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'].values

    # 操作欄位全名列表
    c620_op_col = c620_op.columns.tolist()
    c660_op_col = c660_op.columns.tolist()
    c670_op_col = c670_op.columns.tolist()
    
    # 目標函數 
    def objective(trial):
      
      # 1.調整c620的操作
      c620_op_opt_dict = {}
      for name in c620_op_col:
        if name in self.c620_op_col_can_change:
          c620_op_opt_dict[name] = trial.suggest_uniform(name,self.c620_op_min[name],self.c620_op_max[name])
        else:
          c620_op_opt_dict[name] = c620_op[name].values[0] 
      c620_op_opt = pd.DataFrame(c620_op_opt_dict,index=idx)

      # 2.調整c660的操作
      c660_op_opt_dict = {}
      for name in c660_op_col:
        if name in self.c660_op_col_can_change:
          c660_op_opt_dict[name] = trial.suggest_uniform(name,self.c660_op_min[name],self.c660_op_max[name])
        else:
          c660_op_opt_dict[name] = c660_op[name].values[0] 
      c660_op_opt = pd.DataFrame(c660_op_opt_dict,index=idx)

      # 3.在dist_rate這個欄位上做調整,根據廠區需求 dist rate 愈小愈好 範圍落在[0,10.25]
      dist_rate = trial.suggest_float('Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr',0,10.25)
      c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = dist_rate
      
      # 4.計算c620_wt
      c620_sp = self.c620_F.predict(c620_case.join(c620_feed).join(c620_op_opt))
      s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
      w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
      wt = np.hstack((w1,w2,w3,w4))
      c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
      
      # 5.從c620_wt預測結果計算bz誤差(輸入端和輸出端要一致)
      輸入端bz = c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'].values[0]
      輸出端bz = c620_wt['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%'].values[0]
      loss1 = abs(輸入端bz - 輸出端bz) / 輸入端bz  
      
      # 6.鼓勵dist_rate接近於一個非常小的值
      loss2 = abs(dist_rate - self.e) / dist_rate

      # 7.計算c660_combined_feed_wt
      V615_Btm_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr'].values.reshape(-1,1) #體積流量
      C820_Dist_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr'].values.reshape(-1,1) #體積流量
      V615_Btm_ton = V615_Btm_m3*self.V615_density # 體積流量轉重量流量
      C820_Dist_ton = C820_Dist_m3*self.C820_density # 體積流量轉重量流量
      c620_feed_rate_ton = V615_Btm_ton+C820_Dist_ton # 兩股油相加得到c620總入料重量流量
      c620_mf_side = np.sum(c620_feed_rate_ton*c620_feed.values*s3*0.01,axis=1,keepdims=True) # c620_side流量
      c620_mf_bot = np.sum(c620_feed_rate_ton*c620_feed.values*s4*0.01,axis=1,keepdims=True) # c620_bott流量
      t651_mf = (icg_input['Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr']*self.T651_density).values.reshape(-1,1) # t651重量流量
      c660_mf = t651_mf + c620_mf_side # 兩股相加得到c660入料重量流量
      t651_mf_p ,c620_mf_side_p = t651_mf/c660_mf ,c620_mf_side/c660_mf # 兩股油源的占比百分比
      c660_feed = c620_wt[self.c620_col['sidedraw_x']].values*c620_mf_side_p + t651_feed.values*t651_mf_p #計算c660入料組成
      c660_feed = pd.DataFrame(c660_feed,index=idx,columns=self.c660_col['x41'])

      # 8.取代ICG功能 歷史數據中NA in Benzene_ppmw範圍落在 [800~980] 在這個範圍內搜索即可
      輸入端nainbz =  trial.suggest_float('Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw',800,980)
      c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'] = 輸入端nainbz
      
      # 9.這裡設定輸入端tol為一個數值,可以是使用者給定或是開啟auto_set_x0 = True 則自動設置為10
      輸入端tol = icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'].values[0]
      
      # 10.計算c660_wt
      c660_sp = self.c660_F.predict(c660_case.join(c660_feed).join(c660_op_opt))
      s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
      w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
      wt = np.hstack((w1,w2,w3,w4))
      c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])

      # 11.根據預測的c660_wt 計算誤差 NA in Benzene_ppmw誤差 和 Toluene in Benzene_ppmw 誤差
      na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
      輸出端nainbz = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000
      輸出端tol = c660_wt['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000
      loss3 = abs(輸入端nainbz - 輸出端nainbz) / 輸入端nainbz
      loss4 = abs(輸入端tol - 輸出端tol) / 輸入端tol

      # 誤差加總返回
      return loss1 + loss2 + loss3 + loss4
    
    # cma-es給定優化初始值x0
    x0 = {}
    for name in c620_op_col:
      x0[name] = c620_op[name].values[0]
    for name in c660_op_col:
      x0[name] = c660_op[name].values[0]
    x0['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values[0]
    x0['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values[0]
    
    # cma-es 開始優化
    sampler = optuna.samplers.CmaEsSampler(x0=x0)
    study = optuna.create_study(sampler=sampler)
    study.optimize(objective, n_trials = search_iteration)
    
    # 優化完畢給出可能是最優的參數
    params_df = pd.DataFrame(study.best_params,index=idx,columns=
                             c620_op_col+c660_op_col+
                             ['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr',
                              'Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']
                             )
    
    # 1.最優的c620_op操作 
    c620_op_opt = c620_op.drop(self.c620_op_col_can_change,axis=1).join(params_df[c620_op_col][self.c620_op_col_can_change])
    # 2.最優的c660_op操作
    c660_op_opt = c660_op.drop(self.c660_op_col_can_change,axis=1).join(params_df[c660_op_col][self.c660_op_col_can_change])
    # 3.最優的dist rate
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = params_df['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr']
    # 4.最優的NA in Benzene_ppmw
    c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'] = params_df['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']
    
    # 計算c620_wt
    c620_sp = self.c620_F.predict(c620_case.join(c620_feed).join(c620_op_opt))
    s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
    
    # 計算bz誤差
    輸入端bz = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values[0]
    輸出端bz = c620_wt['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%'].values[0]
    bz_error = abs(輸入端bz-輸出端bz)
    
    # 計算c660_feed
    V615_Btm_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr'].values.reshape(-1,1)
    C820_Dist_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr'].values.reshape(-1,1)
    V615_Btm_ton = V615_Btm_m3*self.V615_density # 體積流量轉重量流量
    C820_Dist_ton = C820_Dist_m3*self.C820_density # 體積流量轉重量流量
    c620_feed_rate_ton = V615_Btm_ton+C820_Dist_ton # 兩股相加得到c620總入料重量流量
    c620_mf_side = np.sum(c620_feed_rate_ton*c620_feed.values*s3*0.01,axis=1,keepdims=True) # c620_side 流量
    c620_mf_bot = np.sum(c620_feed_rate_ton*c620_feed.values*s4*0.01,axis=1,keepdims=True) # c620_bott 流量
    t651_mf = (icg_input['Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr']*self.T651_density).values.reshape(-1,1) # t651重量流量
    c660_mf = t651_mf + c620_mf_side # 兩股相加得到c660入料重量流量
    t651_mf_p ,c620_mf_side_p = t651_mf/c660_mf ,c620_mf_side/c660_mf # 兩股油源的占比百分比
    c660_feed = c620_wt[self.c620_col['sidedraw_x']].values*c620_mf_side_p + t651_feed.values*t651_mf_p #計算c660入料組成
    c660_feed = pd.DataFrame(c660_feed,index=idx,columns=self.c660_col['x41'])

    # 計算c660_wt
    c660_sp = self.c660_F.predict(c660_case.join(c660_feed).join(c660_op_opt))
    s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])
    
    # 計算nainbz誤差和tol誤差
    輸入端nainbz = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values[0]
    輸入端tol = icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'].values[0]
    na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
    輸出端nainbz = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000
    輸出端tol = c660_wt['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000
    nainbz_error = abs(輸入端nainbz-輸出端nainbz)
    tol_error = abs(輸入端tol-輸出端tol)
    
    # 打印優化結果
    print('bz_error:',bz_error)
    print('nainbz_error:',nainbz_error)
    print('tol_error:',tol_error)
    print('dist_rate',c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values[0])
    print('nainbz:',c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000)

    # c670 部份
    c660_mf_bot = np.sum(c660_mf*c660_feed.values*s4*0.01,axis=1,keepdims=True) # c660_bot 重量流量
    c670_mf = c620_mf_bot + c660_mf_bot # c620_bot 重量流量 加上 c660_bot 重量流量 得到c670總入料重量流量
    c620_mf_bot_p,c660_mf_bot_p = c620_mf_bot/c670_mf , c660_mf_bot/c670_mf # 兩股油源的百分比占比
    c670_feed = c620_wt[self.c620_col['bottoms_x']].values*c620_mf_bot_p + c660_wt[self.c660_col['bottoms_x']].values*c660_mf_bot_p # c670入料組成
    c670_feed = pd.DataFrame(c670_feed,index=idx,columns=self.c670_col['combined'])
    
    # c670 upper_bf 計算
    c670_bf = pd.DataFrame(index=idx,columns=self.c670_col['upper_bf']) 
    c620_bot_x = c620_wt[self.c620_col['bottoms_x']].values
    c660_bot_x = c660_wt[self.c660_col['bottoms_x']].values
    upper_bf = (c660_bot_x*c660_mf_bot)/(c620_bot_x*c620_mf_bot+c660_bot_x*c660_mf_bot)
    upper_bf = pd.DataFrame(upper_bf,index=idx,columns=self.c670_col['upper_bf'])
    upper_bf[list(set(self.index_9999)&set(upper_bf.columns))] = 0.9999
    upper_bf[list(set(self.index_0001)&set(upper_bf.columns))] = 0.0001
    
    # c670因為沒有指定wt數值要多少所以直接預測操作條件和分離係數即可
    c670_output = self.c670_M.predict(c670_feed.join(upper_bf))
    c670_sp,c670_op_opt = c670_output.iloc[:,:41*2],c670_output.iloc[:,41*2:]    
    
    # 計算c670_wt
    s1 = c670_sp[self.c670_col['distillate_sf']].values
    s2 = c670_sp[self.c670_col['bottoms_sf']].values
    w1 = sp2wt(c670_feed,s1)
    w2 = sp2wt(c670_feed,s2)
    c670_wt = np.hstack((w1,w2))
    c670_wt = pd.DataFrame(c670_wt,index = idx,columns=self.c670_col['distillate_x']+self.c670_col['bottoms_x'])
    
    # 是否修正操作條件 for 現場數據
    if real_data_mode == False:
      return c620_wt,c620_op_opt,c660_wt,c660_op_opt,c670_wt,c670_op_opt,bz_error,nainbz_error,tol_error
    
    if real_data_mode == True:
      # 有些欄位現場數據沒有 因此drop掉
      c620_op_col = c620_op.drop(['Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      c660_op_col = c660_op.drop(['Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      c670_op_col = c670_op.drop(['Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      
      # 經過修正模組修正op
      op_pred = self.op_fix_model(torch.cat((
        torch.FloatTensor(c620_op_opt[c620_op_col].values),
        torch.FloatTensor(c660_op_opt[c660_op_col].values),
        torch.FloatTensor(c670_op_opt[c670_op_col].values)),dim=1))  
      op_pred = pd.DataFrame(op_pred.detach().numpy(),index=idx)
      
      # 新的op
      new_c620_op = op_pred.iloc[:,:8]
      new_c660_op = op_pred.iloc[:,8:16]
      new_c670_op = op_pred.iloc[:,-5:]
      new_c620_op.columns = c620_op_col
      new_c660_op.columns = c660_op_col
      new_c670_op.columns = c670_op_col
      
      # 更新op
      c620_op_opt.update(new_c620_op)
      c660_op_opt.update(new_c660_op)
      c670_op_opt.update(new_c670_op)
      
      return c620_wt,c620_op_opt,c660_wt,c660_op_opt,c670_wt,c670_op_opt,bz_error,nainbz_error,tol_error

# 試算模式測試 觀察重點 準

In [258]:
f = AllSystem(config)
idx = np.random.choice(allidx,size=100,replace=False,p=None)
# minibatch input 
icg_input = icg_input.loc[idx]
c620_feed = c620_feed.loc[idx]
t651_feed = t651_feed.loc[idx]
# minibatch output 
c620_op = c620_op.loc[idx]
c620_wt = c620_wt.loc[idx]
c660_op = c660_op.loc[idx]
c660_wt = c660_wt.loc[idx]
c670_op = c670_op.loc[idx]
c670_wt = c670_wt.loc[idx]
c620_wt_,c620_op_,c660_wt_,c660_op_,c670_wt_,c670_op_ = f.inference(icg_input,c620_feed,t651_feed)
f.c670_M.show_metrics(c620_wt,c620_wt_,e=2e-2)

,R2,MSE,MAPE
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.886739,0.0024208,2.24028
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.890262,0.549378,1.59932
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.595875,0.108755,0.509682
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.867933,0.234881,4.82946
...,...,...,...
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.999411,1.70613e-06,0.281703
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.9994,3.14532e-07,0.281702
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [259]:
f.c670_M.show_metrics(c620_op,c620_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.999829,2.51688e-10,0.0014344
Density_Vent Gas Production Rate and Composition,0.953963,1.12161e-06,0.188662
Density_Distillate Production Rate and Composition,0.858262,1.75969e-05,0.474913
Density_Sidedraw Production Rate and Composition,0.995049,5.34052e-09,0.00638485
Density_Bottoms Production Rate and Composition,0.99705,5.06583e-11,0.000685382
Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,0.982254,2.77703,0.892007
Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.986565,0.017926,0.755077
Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.985179,0.0288008,0.775988
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,0.984137,0.0199587,0.0628044
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC,0.988609,0.00800199,0.0361974


In [260]:
f.c670_M.show_metrics(c660_wt,c660_wt_,e=2e-2)

,R2,MSE,MAPE
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.78306,0.0498007,13.826
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.910527,3.59415,6.74992
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.634234,2.6219,5.77067
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.896311,1.18839,14.679
...,...,...,...
Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.963479,5.50113e-25,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.964703,6.44645e-31,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [261]:
f.c670_M.show_metrics(c660_op,c660_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.988759,8.69272e-09,0.00782623
Density_Vent Gas Production Rate and Composition,0.932161,2.22601e-05,0.74691
Density_Distillate (Benzene Drag) Production Rate and Composition,0.942908,4.18903e-06,0.197395
Density_Sidedraw (Benzene )Production Rate and Composition,0.963344,2.17298e-12,0.000139871
Density_Bottoms Production Rate and Composition,0.99443,3.99265e-10,0.00178885
Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,0.969874,27.2591,3.46305
Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.968738,0.218335,3.13253
Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.971783,0.212252,3.22522
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,0.973789,0.00119531,0.0324423
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC,0.963515,0.0297057,0.148502


In [262]:
f.c670_M.show_metrics(c670_wt,c670_wt_,e=2e-2)

,R2,MSE,MAPE
Toluene Column C670 Operation_Distillate Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Methane_wt%,0,5.93416e-10,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Ethane_wt%,0,3.60888e-07,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Propane_wt%,0,3.03571e-07,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_n-Butane_wt%,0,5.96216e-47,NaN
...,...,...,...
Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.999999,2.75385e-08,0.0196678
Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.999999,5.05845e-09,0.0197297
Toluene Column C670 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Toluene Column C670 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [263]:
f.c670_M.show_metrics(c670_op,c670_op_,e=2e-2)

,R2,MSE,MAPE
Density_Distillate Production Rate and Composition,0.993221,5.15265e-12,0.00020614
Density_Bottoms Production Rate and Composition,0.993692,1.15915e-09,0.00278305
Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,0.981596,12.5363,1.12303
Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.98463,0.146894,1.10294
Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.983745,0.149123,1.0968
Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,0.967036,0.00239367,0.0212086
Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC,0.988656,0.0315141,0.0617163
AVG,0.984654,1.83803,0.486955


# 確認三項條件有滿足

In [264]:
a = icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]
b = c620_wt[['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]
a.join(b)

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
060-014,80.0,79.999992
001-002,90.0,90.000000
100-017,80.0,79.999992
073-020,70.0,70.000023
108-026,70.0,70.000008
...,...,...
050-026,70.0,70.000000
133-005,90.0,90.000015
098-008,90.0,90.000000
093-020,70.0,70.000008


In [265]:
a = icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]
b = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,[1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] ].sum(axis=1)*10000
b.name = 'c660_wt NA in NA in Benzene_ppmw'
a.join(b)

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,c660_wt NA in NA in Benzene_ppmw
060-014,950.0,950.000661
001-002,980.0,979.858033
100-017,920.0,919.997709
073-020,980.0,979.968704
108-026,860.0,860.006860
...,...,...
050-026,920.0,919.995475
133-005,950.0,950.002138
098-008,920.0,920.003444
093-020,980.0,979.992209


In [266]:
a = icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]
b = c660_wt[['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%']]*10000
a.join(b)

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%
060-014,9.999993,9.999993
001-002,5.000000,5.000000
100-017,5.000006,5.000006
073-020,4.999987,4.999987
108-026,5.000000,5.000000
...,...,...
050-026,4.999998,4.999998
133-005,5.000000,5.000000
098-008,5.000000,5.000000
093-020,10.000004,10.000004


# 推薦模式測試觀察重點調幅(op_opt - op)以及已經是否滿足約束條件或足夠趨近約束條件

In [267]:
f = AllSystem(config)
idx = 5
icg_input = icg_input.iloc[[idx]]
c620_feed = c620_feed.iloc[[idx]]
t651_feed = t651_feed.iloc[[idx]]
# minibatch output 
c620_op = c620_op.iloc[[idx]]
c620_wt = c620_wt.iloc[[idx]]
c660_op = c660_op.iloc[[idx]]
c660_wt = c660_wt.iloc[[idx]]
c670_op = c670_op.iloc[[idx]]
c670_wt = c670_wt.iloc[[idx]]

print(icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'])
print(icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'])
print(icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'])

120-005    90.0
Name: Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%, dtype: float64
120-005    860.0
Name: Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw, dtype: float64
120-005    10.000108
Name: Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw, dtype: float64


In [268]:
print(icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'])

120-005    0.01
Name: Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr, dtype: float64


In [269]:
c620_wt,c620_op_opt,c660_wt,c660_op_opt,c670_wt,c670_op_opt,bz_error,nainbz_error,tol_error = f.recommend(icg_input,c620_feed,t651_feed,search_iteration=300,auto_set_x0=True)

[I 2021-04-26 14:35:36,003] A new study created in memory with name: no-name-48f78a48-7e66-4629-a2f6-de93da61e037
[I 2021-04-26 14:35:36,175] Trial 0 finished with value: 2.8329616758213323 and parameters: {'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 177.84667658970588, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 193.83582050086358, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC': 87.17822206712508, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC': 92.16437878616145, 'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr': 5.997202773790919, 'Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw': 922.3115857329672}. Best is trial 0 with value: 2.8329616758213323.


===================auto_set_x0=====================================
icg_input[Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%] = 70
icg_input[Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw] = 980
icg_input[Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw] = 10


[I 2021-04-26 14:35:36,333] Trial 1 finished with value: 1.4550172803965156 and parameters: {'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 175.47661826411783, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 192.1227853042159, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC': 86.7896704027466, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC': 90.92841569451963, 'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr': 0.05104367556845184, 'Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw': 979.9114310380852}. Best is trial 1 with value: 1.4550172803965156.
[I 2021-04-26 14:35:36,493] Trial 2 finished with value: 1.281791190819049 and parameters: {'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 175.54094210119968, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 192.2

bz_error: 0.6955690292006551
nainbz_error: 117.85597248465831
tol_error: 0.009668527918893943
dist_rate 0.649788168873279
nainbz: 862.1440275153417


# 確認三項條件有滿足

In [270]:
bz_error,nainbz_error,tol_error

(0.6955690292006551, 117.85597248465831, 0.009668527918893943)

In [271]:
a = icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]
b = c620_wt[['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]
a.join(b)

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
120-005,90.0,90.695569


In [272]:
a = icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]
b = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,[1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] ].sum(axis=1)*10000
b.name = 'c660_wt NA in NA in Benzene_ppmw'
a.join(b)

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,c660_wt NA in NA in Benzene_ppmw
120-005,980,862.144028


In [273]:
a = icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]
b = c660_wt[['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%']]*10000
a.join(b)

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%
120-005,10,10.009669


# 確認操作條件有足夠大的調幅

In [274]:
c620_op_opt - c620_op

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate Production Rate and Composition,Density_Sidedraw Production Rate and Composition,Density_Bottoms Production Rate and Composition,Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
120-005,-0.000006,-0.000613,0.003094,-0.000043,0.000004,-5.893219,-0.332297,-0.519026,0.354694,-0.455988


In [275]:
c660_op_opt - c660_op

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate (Benzene Drag) Production Rate and Composition,Density_Sidedraw (Benzene )Production Rate and Composition,Density_Bottoms Production Rate and Composition,Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
120-005,0.000004,0.002998,-0.002246,4.705602e-08,0.000006,0.200943,0.002806,-0.001114,-0.087043,-0.19972


In [276]:
c670_op_opt - c670_op

,Density_Distillate Production Rate and Composition,Density_Bottoms Production Rate and Composition,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
120-005,-7.938627e-07,-0.000006,-6.500122,-0.875822,-0.989508,-0.023361,-0.286453


# 確定以下三點誤差足夠小

In [277]:
bz_error

0.6955690292006551

In [278]:
nainbz_error

117.85597248465831

In [279]:
tol_error

0.009668527918893943

# 保存樣本

In [ ]:
idx = icg_input.index
print(idx)

Index(['002-002'], dtype='object')


In [ ]:
demo = {
      # input
      'icg_input':icg_input.loc[idx],
      'c620_feed':c620_feed.loc[idx],
      't651_feed':t651_feed.loc[idx],
      # output
      'c620_op':c620_op.loc[idx],
      'c620_wt':c620_wt.loc[idx],
      'c660_op':c660_op.loc[idx],
      'c660_wt':c660_wt.loc[idx],
      'c670_op':c670_op.loc[idx],
      'c670_wt':c670_wt.loc[idx],
      }

In [ ]:
import joblib
joblib.dump(demo,"/content/drive/MyDrive/台塑輕油案子/data/c620/demo/demo.pkl")

['/content/drive/MyDrive/台塑輕油案子/data/c620/demo/demo.pkl']

# 保存模組

In [ ]:
joblib.dump(f,"/content/drive/MyDrive/台塑輕油案子/data/c620/model/allsystem.pkl")

['/content/drive/MyDrive/台塑輕油案子/data/c620/model/allsystem.pkl']